In [166]:
import numpy as np
from Jcb import Jcb, calc_basis_matrix
from scipy.linalg import block_diag
from tqdm import tqdm
from scipy.optimize import minimize

In [167]:
p = 2
runs = 4
Ky = 3
Kx = [[4]]
Kb = [[2]]

In [193]:
def calc_I_theta(p, Ky):
    return np.eye((p+1)*Ky, dtype=float)
def calc_J_CH(Kx, Kb):
    J_chs = [Jcb(*[calc_basis_matrix(x_basis=x, b_basis=b) for x, b in zip(x_row, b_row)]) for x_row, b_row in zip(Kx, Kb)]
    bases = [1] + J_chs
    return block_diag(*bases)
def calc_Sigma(p, Ky, N, decay=0):
    In = np.eye(N)
    # initialization for decay==0
    sigma1 = np.eye((p+1)*Ky)
    if decay==1:
        sigma1 = np.eye((p+1)*Ky)
    return np.kron(In, sigma1)
def Covar(Gamma, J_CH, I_theta, Sigma):
    Z = Gamma @ J_CH
    ZtZ_inv = np.linalg.inv(Z.T @ Z)
    L1 = ZtZ_inv @ Z.T
    L = np.kron(I_theta, L1)
    return L @ Sigma @ L.T
def optimality(Var, type='A'):
    if type=='A':
        return np.trace(Var)
    elif type =='D':
        return np.linalg.det(Var)

def cordex_func(runs, Kx,
                I_theta, J_CH, Sigma, epochs=1_000):

    def objective(x):
        Gamma_[run, feat] = x
        Gamma = np.hstack((np.ones((runs, 1)), Gamma_))
        Z = Gamma @ J_CH
        ZtZ_inv = np.linalg.inv(Z.T @ Z)
        L1 = ZtZ_inv @ Z.T
        L = np.kron(I_theta, L1)
        Covar = L @ Sigma @ L.T
        return np.trace(Covar)

    Best_des = None
    Best_obj = np.inf

    for _ in tqdm(range(epochs)):
        objective_value = np.inf
        Gamma_ = np.hstack([np.random.rand(runs, k[0]) for k in Kx])

        for run in range(runs):
            for feat in range(np.sum(Kx)-1):
                res = minimize(objective, Gamma_[run, feat], method='L-BFGS-B', bounds=[(-1., 1.)])
                if res.x is not None:
                    Gamma_[run, feat] = res.x
                objective_value = objective(res.x)
        if 0 <= objective_value < Best_obj:
            Best_obj = objective_value
            Best_des = Gamma_

    return Best_des, np.abs(Best_obj)

In [194]:
I_theta = calc_I_theta(p=p, Ky=Ky)
J_CH = calc_J_CH(Kx=Kx, Kb=Kb)
Sigma = calc_Sigma(p=p, Ky=Ky, N=runs, decay=0)
Gamma = np.hstack([np.ones((runs, 1))] + [np.random.rand(runs, k[0]) for k in Kx])
Var = Covar(Gamma=Gamma, J_CH=J_CH, I_theta=I_theta, Sigma=Sigma)

In [195]:
optimality(Var, type='A')

3550.274705003238

In [196]:
Des, Cr = cordex_func(runs=runs,
                      Kx=Kx,
                      I_theta=I_theta,
                      J_CH=J_CH,
                      Sigma=Sigma,
                      epochs=1_000)

100%|██████████| 1000/1000 [00:09<00:00, 100.31it/s]


In [199]:
np.round(Des)

array([[-1., -1., -1.,  0.],
       [ 1.,  1., -1.,  0.],
       [-1.,  0.,  1.,  1.],
       [ 1.,  1., -1.,  0.]])

In [198]:
Cr

132.48516906911735